<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/CleanedData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.utils import resample

# STEP 1: Loading Dataset

# Use the raw URL for the CSV file on GitHub
url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
df = pd.read_csv(url)

# ------------------------------------------------------------------------------

# STEP 2: Data Cleaning

# Strip leading/trailing whitespaces in column names
df.columns = df.columns.str.strip()

# Remove rows with any missing values
df = df.dropna()  # Drop rows with NaN (e.g. 'Flow Bytes/s')

# Drop columns with constant values (no variance)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=constant_cols)

# Remove duplicated rows (if any)
df = df.drop_duplicates()

# Encode 'Label' column: 'DDoS' as 1, 'BENIGN' as 0
df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)

# Show cleaned dataset info
print("="*50)
print(" AFTER CLEANING")
print("="*50)
print(f"Total rows after cleaning: {len(df)}")
print("Label distribution after cleaning:")
print(df['Label'].value_counts())
print("="*50)
df.head()

# ------------------------------------------------------------------------------

# STEP 3: Class Balancing with Undersampling


# Separate majority (DDoS) and minority (BENIGN) classes
df_majority = df[df['Label'] == 1]  # DDoS
df_minority = df[df['Label'] == 0]  # BENIGN

print("Class Distribution Before Balancing:")
print(df['Label'].value_counts())

# Downsample the majority class (DDoS) to match the number of benign samples
df_majority_downsampled = resample(df_majority,
                                   replace=False,            # sample without replacement
                                   n_samples=len(df_minority),  # match minority count
                                   random_state=42)           # reproducibility

# Combine downsampled majority with minority class
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the combined dataframe
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Show updated class balance
print("\n AFTER UNDERSAMPLING")
print("="*50)
print(f"Total rows after balancing: {len(df_balanced)}")
print("Class balance:")
print(df_balanced['Label'].value_counts())
print("="*50)


# ------------------------------------------------------------------------------

# STEP 4: Display Sample Data (From Balanced Dataset)

# Columns to show in sample output
columns_to_show = ['Destination Port', 'Flow Duration', 'Label']

# Count label values (from balanced data)
ddos_count = df_balanced[df_balanced['Label'] == 1].shape[0]
benign_count = df_balanced[df_balanced['Label'] == 0].shape[0]
total_rows = df_balanced.shape[0]

print("\n" + "="*60)
print("SAMPLE OF BALANCED DATAFRAME")
print("="*60)
print(f"Total Rows     : {total_rows}")
print(f"DDoS Attacks   : {ddos_count}")
print(f"Benign Records : {benign_count}")
print("="*60)

# Display sample rows
print("\n Balanced DataFrame (First 5 Rows):")
display(df_balanced[columns_to_show].head(5))

print("\n DDoS Samples (First 5):")
display(df_balanced[df_balanced['Label'] == 1][columns_to_show].head(5))

print("\n Benign Samples (First 5):")
display(df_balanced[df_balanced['Label'] == 0][columns_to_show].head(5))

# ------------------------------------------------------------------------------

# STEP 5: Extract Statistical Features (on Balanced Data)

# Create a trimmed preview column from flow characteristics
df_balanced['Flow Preview'] = df_balanced[['Destination Port', 'Flow Duration']].astype(str).agg(' | '.join, axis=1)

# Feature 1: Packet length difference
df_balanced['pkt_length_diff'] = df_balanced['Max Packet Length'] - df_balanced['Min Packet Length']

# Feature 2: Packet length variation (Max / Mean)
df_balanced['pkt_length_var_ratio'] = (df_balanced['Max Packet Length'] / (df_balanced['Packet Length Mean'] + 1e-5)).round(3)

# Feature 3: Byte ratio (Fwd vs Bwd)
df_balanced['byte_ratio'] = (df_balanced['Total Length of Fwd Packets'] / (df_balanced['Total Length of Bwd Packets'] + 1e-5)).round(3)

# Feature 4: Flow duration per packet
df_balanced['duration_per_packet'] = (df_balanced['Flow Duration'] / (df_balanced['Total Fwd Packets'] + df_balanced['Total Backward Packets'] + 1e-5)).round(3)

# Feature 5: Average packet size vs Max packet size
df_balanced['avg_to_max_ratio'] = (df_balanced['Average Packet Size'] / (df_balanced['Max Packet Length'] + 1e-5)).round(3)

# Remove duplicated columns (just in case)
df_balanced = df_balanced.loc[:, ~df_balanced.columns.duplicated()]

# Preview extracted features
print("\n Sample Extracted Features (First 5):")
feature_cols = ['Flow Preview', 'pkt_length_diff', 'pkt_length_var_ratio',
                'byte_ratio', 'duration_per_packet', 'avg_to_max_ratio']
display(df_balanced[feature_cols].head(5))

# Show final column list
print("\n Final Columns:")
print(df_balanced.columns.tolist())


/tmp/ipython-input-1811194195.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)


 AFTER CLEANING
Total rows after cleaning: 223108
Label distribution after cleaning:
Label
1    128016
0     95092
Name: count, dtype: int64
Class Distribution Before Balancing:
Label
1    128016
0     95092
Name: count, dtype: int64

 AFTER UNDERSAMPLING
Total rows after balancing: 190184
Class balance:
Label
1    95092
0    95092
Name: count, dtype: int64

SAMPLE OF BALANCED DATAFRAME
Total Rows     : 190184
DDoS Attacks   : 95092
Benign Records : 95092

 Balanced DataFrame (First 5 Rows):


,Destination Port,Flow Duration,Label
0,80,525915,1
1,53,173,0
2,80,33700,1
3,53,295753,0
4,443,117168001,0



 DDoS Samples (First 5):


,Destination Port,Flow Duration,Label
0,80,525915,1
2,80,33700,1
9,80,624202,1
10,80,2603947,1
11,80,70721898,1



 Benign Samples (First 5):


,Destination Port,Flow Duration,Label
1,53,173,0
3,53,295753,0
4,443,117168001,0
5,53,31435,0
6,80,11,0



 Sample Extracted Features (First 5):


,Flow Preview,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,80 | 525915,5840,5.020,0.002,58434.935,0.221
1,53 | 173,16,1.194,0.729,43.250,1.047
2,80 | 33700,5840,5.020,0.002,3744.440,0.221
3,53 | 295753,103,2.091,0.539,49292.085,0.558
4,443 | 117168001,1430,7.575,0.147,4686718.165,0.137



 Final Columns:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd 